In [1]:
# General imports 
import cv2
import numpy as np
import json

In [2]:
# Analysis parameters
with open('../../../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# Define directories 
home_dir = '/Users/uriel/disks/meso_shared'
main_dir = '{}/RetinoMaps'.format(home_dir)


tasks = ['pRF', 'PurLoc', 'SacLoc']
task = tasks[0]

stim_video_fn = '{}/derivatives/vdm/{}_vid.mp4'.format(main_dir,task)



In [3]:
# open video
cap = cv2.VideoCapture(stim_video_fn)

# get video settings
vid_width, vid_height = cap.get(3), cap.get(4)
vid_fps = cap.get(cv2.CAP_PROP_FPS)
vid_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# set VDM settings
TR = analysis_info['TR']
vdm_width = analysis_info['vdm_size_pix_eyes_mvt'][0] 
vdm_height = analysis_info['vdm_size_pix_eyes_mvt'][1]
vdm_frames_sampled = vid_fps * TR
vdm_frames = vid_frames / vdm_frames_sampled
vdm_mat = np.zeros((vdm_width, vdm_height, int(vdm_frames)))

# define top and bottom part as 16/9 screen experiment and VDM should be a square
height_to_add = (vid_width - vid_height) / 2 
add_mat =  np.zeros((int(height_to_add),int(vid_width)))

In [4]:
# defind outputs fn 
vdm_video_fn = '{}/derivatives/vdm/vdm_{}_{}_{}.mp4'.format(main_dir, task, vdm_width, vdm_height)
vdm_numpy_fn = '{}/derivatives/vdm/vdm_{}_{}_{}.npy'.format(main_dir, task, vdm_width, vdm_height)

In [5]:
#hide the central fixation point
center_x= int(vid_width / 2)
center_y = int(vid_height / 2)
radius = 100

In [6]:
# create VDM matrix
vid_idx, vdm_idx = 0, 0
while(cap.isOpened()):
    
    # read video frames
    ret, vid_frame = cap.read()
    if not ret: break

    # process sampled frames
    if np.mod(vid_idx, int(vdm_frames_sampled)) == 0:

        # convert frame to grayscale
        gray_mat = cv2.cvtColor(vid_frame, cv2.COLOR_BGR2GRAY)

        # convert to binary
        binary_mat = (gray_mat > 5).astype(np.uint8)
    
        # add top and bottom blank    
        binary_reshape_mat = np.concatenate((add_mat,binary_mat,add_mat), axis=0)
        
        # resize to create VDM
        binary_reshape_resize_mat = cv2.resize(binary_reshape_mat, dsize=(vdm_width, vdm_height),
                                               interpolation=cv2.INTER_NEAREST)
        
        # fill VDM matrix
        vdm_mat[...,vdm_idx] = binary_reshape_resize_mat
        vdm_idx += 1


    vid_idx += 1
    
cap.release()

In [63]:
# save VDM as numpy matrix
np.save(vdm_numpy_fn, vdm_mat)

# save VDM as video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(vdm_video_fn, fourcc, 1/TR, (vdm_width, vdm_height), False)
[out.write(np.uint8(frame*255)) for frame in np.split(vdm_mat, vdm_frames, axis=2)]
out.release()